In [2]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


In [3]:
from src.utils import astype_to_category, convert_category_into_integer
from src.model.decision_tree import decision_tree
from src.model.logistic_regression import logit
from src.model.svm import *

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as ds
import torchvision.transforms as transforms

import lightning as L
from lightning.pytorch.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import xgboost as xgb

import warnings

In [4]:
## 경고 무시
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
## train, test 데이터 불러오기
origin_train = pd.read_csv('../data/train.csv', encoding='cp949').dropna()
origin_test = pd.read_csv('../data/test.csv', encoding='cp949')
drop_columns = ['HandsetRefurbished', 'HandsetWebCapable', 'ServiceArea', 'ChildrenInHH', 'TruckOwner', 'RVOwner', 'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'NewCellphoneUser', 'NotNewCellphoneUser', 'NonUSTravel', 'MadeCallToRetentionTeam', 'OwnsMotorcycle', 'CreditRating', 'MaritalStatus', 'Occupation', 'PrizmCode']
origin_train = origin_train.drop(columns=drop_columns)
origin_test = origin_test.drop(columns=drop_columns)
# filter_columns = ['Churn', 'MonthlyRevenue', 'TotalRecurringCharge', 'PercChangeRevenues', 'IncomeGroup', 'MonthlyMinutes', 'DroppedCalls',\
#                    'BlockedCalls', 'OverageMinutes', 'RoamingCalls', 'CustomerCareCalls', 'RetentionCalls', 'MonthsInService',\
#                       'HandsetRefurbished', 'HandsetWebCapable', 'Homeownership', 'NewCellphoneUser', 'AgeHH1', \
#                         'AgeHH2', 'CreditRating', 'AdjustmentsToCreditRating', 'HasCreditCard']
# origin_train = origin_train.filter(items=filter_columns)
# origin_train = origin_train.filter(items=filter_columns)

origin_train = origin_train.query("MonthlyRevenue < 100")
origin_train = origin_train.query("MonthlyMinutes < 1200")
origin_train = origin_train.query("OverageMinutes < 20")
origin_train = origin_train.query("-10 < PercChangeRevenues < 0")
origin_train = origin_train.query("ActiveSubs < 6")
origin_train = origin_train.query("HandsetModels < 4")
origin_train = origin_train.query("MonthsInService < 50")
origin_train = origin_train.query("CustomerCareCalls < 15")

# Step 1: Drop rows with missing values
data_cleaned = origin_train.dropna()
seed = 0

# Step 2: Encoding categorical columns (if necessary)
label_encoders = {}
for column in data_cleaned.select_dtypes(include=['object']).columns:
    if column != 'CustomerID' and column != 'Churn':  # Exclude ID and target column
        label_encoders[column] = LabelEncoder()
        data_cleaned[column] = label_encoders[column].fit_transform(data_cleaned[column])

# Encode target column 'Churn'
le_churn = LabelEncoder()
data_cleaned['Churn'] = le_churn.fit_transform(data_cleaned['Churn'])

# Step 3: Splitting data into features (X) and target (y)
X = data_cleaned.drop(columns=['CustomerID', 'Churn'])
y = data_cleaned['Churn']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

# Step 4: Train and evaluate models
# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=seed)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)

# XGBoost Classifier
xgb_model = xgb.XGBClassifier(random_state=seed)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_predictions)

rf_accuracy, xgb_accuracy


(0.7176738576979766, 0.7149269730671312)